INSTALL INDEPENDICIES

In [ ]:
%pip install stable-baselines3[extra]

In [ ]:
%pip install pygame

In [1]:
import os
import gymnasium as gym
import stable_baselines3
from stable_baselines3 import PPO #PPo is the algorithm we are gonna use here
#those aren't working so sometimes
from stable_baselines3.common.vec_env import dummy_vec_env
from stable_baselines3.common.evaluation import evaluate_policy




Load Env

In [2]:
environment_name='CartPole-v1'
env=gym.make(environment_name,render_mode="human")

In [3]:
import pygame

In [4]:

episodes=5
for episode in range (1,episodes+1):
    state=env.reset()
    done=False
    score=0
    while not done:
        env.render()
        action=env.action_space.sample()
        n_state, reward, done, any, info = env.step(action)
        score += reward
    print('Episode:{} Score{}'.format(episode,score))
env.close()        

Episode:1 Score9.0
Episode:2 Score25.0
Episode:3 Score49.0
Episode:4 Score34.0
Episode:5 Score16.0


Understand the env
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [5]:
env.action_space

Discrete(2)

In [6]:
env.action_space.sample()

1

In [7]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [8]:
env.observation_space.sample()

array([-9.6452290e-01,  6.1042518e+37, -3.8434178e-01, -4.5209329e+37],
      dtype=float32)

Train the RL model

In [9]:
#make your directory manually first
log_path=os.path.join('Training','Logs')

In [10]:
log_path

'Training\\Logs'

In [11]:
env=gym.make(environment_name)
env=stable_baselines3.common.vec_env.DummyVecEnv([lambda:env])
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


You might need to install tensorBoard using 
 conda install -c conda-forge tensorboard

In [15]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 1596 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1033        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007178122 |
|    clip_fraction        | 0.0721      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.509      |
|    explained_variance   | 0.0467      |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0194     |
|    n_updates            | 310         |
|    policy_gradient_loss | -0.00235    |
|    value_loss           | 0.000892    |
-----------------------------------------
---

Save and reload the model

In [16]:
PPo_Path= os.path.join('Training','Saved Models','PPO_Model_CartPole')

In [17]:
model.save(PPo_Path)

In [18]:
del model

In [20]:
PPo_Path

'Training\\Saved Models\\PPO_Model_CartPole'

In [19]:
model=PPO.load(PPo_Path,env=env)

In [21]:
model.learn(total_timesteps=1000)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 1540 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
